In [2]:
from fastembed import TextEmbedding
import numpy as np

In [3]:
MODEL = 'jinaai/jina-embeddings-v2-small-en'

embedding_model = TextEmbedding(MODEL)

1. 

In [4]:
sample_query = 'I just discovered the course. Can I join now?'

In [5]:
dir(embedding_model)

['EMBEDDINGS_REGISTRY',
 'METADATA_FILE',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__firstlineno__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__static_attributes__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_embedding_size',
 '_get_model_description',
 '_list_supported_models',
 '_local_files_only',
 'add_custom_model',
 'cache_dir',
 'decompress_to_cache',
 'download_file_from_gcs',
 'download_files_from_huggingface',
 'download_model',
 'embed',
 'embedding_size',
 'get_embedding_size',
 'list_supported_models',
 'model',
 'model_name',
 'passage_embed',
 'query_embed',
 'retrieve_model_gcs',
 'threads']

In [12]:
results = list(embedding_model.embed([sample_query]))

In [13]:
dir(list[0])

['__add__',
 '__args__',
 '__bases__',
 '__class__',
 '__class_getitem__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mro_entries__',
 '__mul__',
 '__ne__',
 '__new__',
 '__origin__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__typing_unpacked_tuple_args__',
 '__unpacked__',
 'append',
 'clear',
 'copy',
 'count',
 'extend',
 'index',
 'insert',
 'pop',
 'remove',
 'reverse',
 'sort']

In [15]:
len(results[0])

512

In [16]:
# print(len(results[0]))

In [17]:
print(min(results[0]))

-0.11726373551188797


In [18]:
# Q2. Cosine similarity with another vector
doc = 'Can I still join the course after the start date?'

# Embed the document
doc_embedding = list(embedding_model.embed([doc]))[0]

# Get the query embedding (already computed above)
query_embedding = results

# Calculate cosine similarity
def cosine_similarity(v1, v2):
    dot_product = np.dot(v1, v2)
    norm_v1 = np.linalg.norm(v1)
    norm_v2 = np.linalg.norm(v2)
    return dot_product / (norm_v1 * norm_v2)

# Calculate the cosine similarity
cosine_sim = cosine_similarity(query_embedding, doc_embedding)
print(f"Cosine similarity between query and document: {cosine_sim}")

Cosine similarity between query and document: [0.90085289]
